In [1]:
import pandas as pd
import os
import sys

!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/ensemble-boxes-104/ensemble_boxes-1.0.4/ -f ./ --no-index
!pip install ../input/sartoriusensemble/omegaconf-2.1.1-py3-none-any.whl
!pip install ../input/sartoriusensemble/iopath-0.1.9-py3-none-any.whl


Looking in links: ../input/detectron-05/whls
Processing /kaggle/input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=272235 sha256=c84caf069ea9bb6ae10438367466bd7b23ddbdcf85dd1aaf3763b811ead42f61
  Stored in directory: /root/.cache/pip/wheels/d2/09/29/2054c6cc4599cf39f1c52ce3c20bfb50be696caa5d5ba460c9
Successfully built pycocotools
Looking in links: ../input/detectron-05/whls
Processing /kaggle/input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
Processing /kaggle/in

In [2]:
import pandas as pd
import os
import sys
os.system("rm -rf ./*.csv")
csvfiles = []

In [3]:
! python -W ignore ../input/d/charonwangg/test-submit/submit-t.py --modelpath '../input/sub1230/1230_236_207_402.pth' --submitfile 'submit1.csv'
csvfiles.append(pd.read_csv('./submit1.csv'))

Avengers Ensemble!!!!


In [4]:
! python -W ignore ../input/d/charonwangg/test-submit/submit-t.py --modelpath '../input/output-1228-weightedloss-cv328/1228_211_206_400.pth' --submitfile 'submit2.csv'
csvfiles.append(pd.read_csv('./submit2.csv'))

Avengers Ensemble!!!!


In [5]:
! python -W ignore ../input/d/charonwangg/test-submit/submit-t.py --modelpath '../input/sub1229/1229_all_data.pth' --submitfile 'submit3.csv'
csvfiles.append(pd.read_csv('./submit3.csv'))

Avengers Ensemble!!!!


In [6]:
# ! python -W ignore ../input/sartoriusensemble/submit-t.py --modelpath '../input/sub1226/1226_3stage_231_200_397.pth' --submitfile 'submit4.csv'
# csvfiles.append(pd.read_csv('./submit4.csv'))

In [7]:
# ! python -W ignore ../input/sartoriusensemble/submit-t.py --modelpath '../input/weightedloss-1226-217-203-399/1226_217_203_399.pth' --submitfile 'submit5.csv'
# csvfiles.append(pd.read_csv('./submit5.csv'))

In [8]:
THRESHOLDS = [.2664993123909565, .3134462119696301, .6716982906416183]#[0.18744284533097239, 0.18388924150437724, 0.7189045317890984]
MIN_PIXELS = [60, 120, 80]#[30, 160, 60]
PIXEL_CLASS_THRESHOLDS = [0.45, 0.4, 0.45]

In [9]:
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [10]:
from ensemble_boxes import *

def nms_predictions(classes, scores, bboxes, masks, weights=None,
                    iou_th=.5, shape=(520, 704)):
    he, wd = shape[0], shape[1]
    boxes_list = [[x[0] / wd, x[1] / he, x[2] / wd, x[3] / he]
                  for x in bboxes]
    scores_list = [x for x in scores]
    labels_list = [x for x in classes]
    nms_bboxes, nms_scores, nms_classes = nms(
        boxes=[boxes_list], 
        scores=[scores_list], 
        labels=[labels_list], 
        weights=None,
        iou_thr=iou_th
    )

    nms_masks = []
    for s in nms_scores:
        nms_masks.append(masks[scores.index(s)])
#     nms_scores, nms_classes, nms_masks, nms_bboxes = \
#     zip(*sorted(zip(nms_scores, nms_classes, nms_masks, nms_bboxes), reverse=True))
    scores_orig = list(nms_scores.copy())
    nms_scores = sorted(nms_scores,reverse=True).copy()
    #nms_scores, nms_classes, nms_masks, nms_bboxes = zip(*sorted(zip(nms_scores, nms_classes, nms_masks, nms_bboxes), reverse=True))
    nms_masks = [nms_masks[scores_orig.index(s)] for s in nms_scores]
    nms_classes = [nms_classes[scores_orig.index(s)] for s in nms_scores]
    nms_bboxes = [nms_bboxes[scores_orig.index(s)] for s in nms_scores]

    return nms_classes, nms_scores, nms_masks, nms_bboxes

def ensemble_pred_masks(masks, pred_class, min_pixels,used):
    result = []
    for i, mask in enumerate(masks):
        mask = mask * (1 - used)
        if mask.sum() >= min_pixels[pred_class]:
            used += mask
            result.append(rle_encode(mask))
    return result

In [11]:
print(len(csvfiles))

3


## P(mask) = P(mask|model)*P(model)

In [12]:
def compute_overlaps_masks(masks1, masks2):
    # '''
    # mask: [instances,Height,Width,]
    # '''
    # flatten masks
    masks1 = np.reshape(masks1, (masks1.shape[0],-1)).astype(np.float32)
    masks2 = np.reshape(masks2, (masks2.shape[0],-1)).astype(np.float32)
    area1 = np.sum(masks1, axis=-1)
    area2 = np.sum(masks2, axis=-1)

    # intersections and union
    intersections = np.dot(masks1, masks2.T)
    union = area1[:,None] + area2[None,:] - intersections
    overlaps = intersections / union

    return overlaps

def pairwise_screening(masks1,masks2,scores,iou_thr=0.1):
  #删去除了高IoU以外的实例
  rest_idx = []
  overlaps = compute_overlaps_masks(masks1,masks2)
  for mm in range(overlaps.shape[0]):
    if np.max(overlaps[mm]) > iou_thr:
      ind = np.argmax(overlaps[mm])
      rest_idx.append(ind)
  return rest_idx
  

def digits2prob(mat):
    mat = np.array(mat)
    deno = 1/np.sum(mat,axis=0)
    mat = np.multiply(mat,deno)
    return mat

In [13]:
# 三细胞 adding
import numpy as np
import torch
from tqdm import tqdm
import re
ids = csvfiles[0].loc[:,'id'].values
totalids = list(set(ids.tolist()))

subm_ids = []
subm_masks = []

for id in tqdm(totalids,total=len(totalids)):
    nmsmasks,nmsclasses,nmsscores,nmsboxes = [],[],[],[]
    pred_classes = []
    for idx,csvfile in enumerate(csvfiles):
        index = csvfile[csvfile['id'] == id].index
        masks = csvfile.loc[index,'masks'].values
        
        pred_classes = csvfile.loc[index,'classes'].values.tolist()
        pred_class = max(set(pred_classes), key=pred_classes.count)

        #model_confidence = model_confidences[idx][pred_class]
        pred_masks = np.zeros((masks.shape[0],520,704))
        scores = (csvfile.loc[index,'scores']).values.tolist()
        boxes = csvfile.loc[index,'bboxes'].values.tolist()

        bboxes = []
        for i in range(len(boxes)):
            box = boxes[i].replace('[','').replace(']','').split(',')
            bboxes.append(list(map(float,box)))

        for i in range(masks.shape[0]):
            pred_masks[i,:,:] = rle_decode(masks[i])

        if idx == 0:
          base_masks = pred_masks.copy()
          base_classes = pred_classes.copy()
          base_scores = scores.copy()
          base_bboxes = bboxes.copy()
          
        else:
          rest_idx = pairwise_screening(base_masks,pred_masks,0.1)
          pred_masks = pred_masks[rest_idx]
          pred_classes = [pred_classes[idx] for idx in rest_idx]
          scores = [scores[idx] for idx in rest_idx]
          bboxes = [bboxes[idx] for idx in rest_idx]

          #update base
          base_classes.extend(pred_classes)
          base_scores.extend(scores)
          base_bboxes.extend(bboxes)
          base_masks = np.append(base_masks,pred_masks,axis=0)
  
          base_classes, base_scores, base_masks, base_bboxes = nms_predictions(
          base_classes, 
          base_scores, 
          base_bboxes,
          base_masks, 
          #weights=None,#model_confidences[:,pred_class],
          iou_th=0.3
        )
          base_classes = list(base_classes)
          base_bboxes = list(base_bboxes)
          base_bboxes = list(base_bboxes)
          base_scores = list(base_scores)
          base_masks = np.array(base_masks)
  
        nmsboxes.extend(bboxes)
        nmsmasks.extend(pred_masks)
        nmsclasses.extend(pred_classes)
        nmsscores.extend(scores)
  
    nmsclasses, nmsscores, nmsmasks,_ = nms_predictions(
        nmsclasses, 
        nmsscores, 
        nmsboxes,
        nmsmasks, 
        #weights=None,#model_confidences[:,pred_class],
        iou_th=0.3
    )
    used = np.zeros((520, 704))
    pred_class = max(set(nmsclasses), key=nmsclasses.count)
    encoded_masks = ensemble_pred_masks(
        nmsmasks, 
        pred_class, 
        min_pixels=MIN_PIXELS,
        used=used
    )
    for enc_mask in encoded_masks:
        subm_ids.append(id)
        subm_masks.append(enc_mask)

submit = pd.DataFrame({
    'id': subm_ids, 
    'predicted': subm_masks
})
submit.to_csv('submission.csv', index=False)

100%|██████████| 3/3 [00:12<00:00,  4.03s/it]


In [14]:
submit

,id,predicted
0,d8bfd1dafdc4,351 15 1054 18 1758 19 2462 21 3166 23 3871 24...
1,d8bfd1dafdc4,706 5 1410 5 2114 6 2818 6 3522 6 4226 7 4930 ...
2,d8bfd1dafdc4,163948 3 164649 8 165352 10 166054 13 166757 1...
3,d8bfd1dafdc4,170 60 874 63 1577 65 2282 64 2986 65 3690 65 ...
4,d8bfd1dafdc4,225005 6 225709 6 226412 8 227116 8 227819 10 ...
...,...,...
345,7ae19de7bc2a,217627 11 218329 15 219033 16 219737 16 220441...
346,7ae19de7bc2a,323137 7 323841 1 323845 4 324545 1 324551 3 3...
347,7ae19de7bc2a,315597 5 316300 1 317003 1 317706 2 318408 4 3...
348,7ae19de7bc2a,79460 8 80162 11 80865 11 81568 11 82272 10 82...


In [15]:
# import matplotlib.pyplot as plt
# import cv2
# submit = pd.read_csv("submission.csv")
# submit = submit.groupby(by="id")
# png = "7ae19de7bc2a"
# submit = submit.get_group(png)

# _, axs = plt.subplots(1,2, figsize=(40,15))
# axs[1].imshow(cv2.imread(f"../input/sartorius-cell-instance-segmentation/test/{png}.png"))
# for i in range(submit.shape[0]):
#     dec = rle_decode(submit.predicted.iloc[i])
#     axs[0].imshow(np.ma.masked_where(dec==0, dec))